# Proximal Policy Optimization

В практической реализации существует два варианта реализации алгоритма PPO:
* выполняет обновление, ограниченное KL, как TRPO, но штрафует KL-расхождение в целевой функции вместо того, чтобы делать его жестким ограничением, и автоматически регулирует коэффициент штрафа в процессе обучения, чтобы он масштабировался соответствующим образом.
* не содержит в целевой функции члена KL-дивергенции и вообще не имеет ограничения. Вместо этого полагается на специализированный клиппинг

<img src="https://spinningup.openai.com/en/latest/_images/math/e62a8971472597f4b014c2da064f636ffe365ba3.svg">

Спойлер: клиппинг - не самое главное в PPO, как это могло показаться на первый взгляд. Алгоритм PPO работает во многом и за счет небольших дополнительных улучшений. Подробнее: https://arxiv.org/pdf/2005.12729.pdf

# Устанавливаем зависимости

In [ ]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !apt -qq update -y
    !apt -qq install swig -y
    !pip -q install box2d-py
    !pip -q install "gymnasium[classic-control, box2d, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [ ]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import gymnasium as gym
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Память

In [ ]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []

    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

# Сеть Actor-Critic

In [ ]:
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden):
        super(ActorCritic, self).__init__()

        # actor: 2 hidden + output
        # self.action_layer = nn.Sequential(..., nn.Softmax(dim=-1))
        # 3 linear layers
        ####### Здесь ваш код ########

        ##############################

        # critic: 2 hidden + output
        # self.value_layer = nn.Sequential(...)
        # 3 linear layers
        ####### Здесь ваш код ########

        ##############################

    def forward(self):
        raise NotImplementedError

    def act(self, state, memory):
        state = torch.from_numpy(state).float().to(device)

        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        action = dist.sample()

        # сохраняем в память: state, action, log_prob(action)
        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(dist.log_prob(action))

        return action.item()

    def evaluate(self, state, action):
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()

        state_value = self.value_layer(state)
        return action_logprobs, torch.squeeze(state_value), dist_entropy


# PPO policy

In [ ]:
class PPO:
    def __init__(self, state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs

        self.policy = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())

        self.MseLoss = nn.MSELoss()

    def update(self, memory):
        # Monte Carlo оценка вознаграждений:
        returns = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            # обнуляем накопленную награду, если попали в терминальное состояние
            if is_terminal:
                discounted_reward = 0
            # discounted_reward =
            ####### Здесь ваш код ########

            ##############################
            returns.insert(0, discounted_reward)

        returns = torch.tensor(returns, dtype=torch.float32).to(device)
        # выполните нормализацию вознаграждений (r - mean(r)) / std(r + 1e-5):
        # returns =
        ####### Здесь ваш код ########

        ##############################

        # конвертация list в tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.stack(memory.logprobs).to(device).detach()

        # оптимизация K epochs:
        for _ in range(self.K_epochs):
            # получаем logprobs, state_values, dist_entropy от стратегии:
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # находим отношение стратегий (pi_theta / pi_theta__old), через logprobs и old_logprobs.detach():
            # ratios =
            ####### Здесь ваш код ########

            ##############################

            # считаем advantages
            # advantages =
            ####### Здесь ваш код ########

            ##############################

            # Находим surrogate loss:
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1 - self.eps_clip, 1 + self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5 * self.MseLoss(state_values, returns) - 0.01 * dist_entropy

            # делаем шаг градиента
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()

        # копируем веса
        self.policy_old.load_state_dict(self.policy.state_dict())

# Основной цикл

In [ ]:
# env_name = "CartPole-v1"
env_name = "LunarLander-v3"
# env_name = "MountainCar-v0"

env = gym.make(env_name)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
render = False
solved_reward = 200  # останавливаемся если avg_reward > solved_reward
log_interval = 20  # печатаем avg reward  в интервале
max_episodes = 50000  # количество эпизодов обучения
max_timesteps = 500  # максимальное кол-во шагов в эпизоде
n_latent_var = 64  # кол-во переменных в скрытых слоях
update_timestep = 2000  # обновляем policy каждые n шагов
lr = 0.001 # learning rate
betas = (0.9, 0.999) # betas для adam optimizer
gamma = 0.99  # discount factor
K_epochs = 4  # количество эпох обновления policy
eps_clip = 0.1  # clip параметр для PPO
random_seed = None

if random_seed:
    torch.manual_seed(random_seed)
    env.seed(random_seed)

memory = Memory()
ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)
print(lr, betas)

# переменные для логирования
running_reward = 0
avg_length = 0
timestep = 0

# цикл обучения
for i_episode in range(1, max_episodes + 1):
    state, _ = env.reset()
    for t in range(max_timesteps):
        timestep += 1

        # используем policy_old для выбора действия
        action = ppo.policy_old.act(state, memory)
        state, reward, terminated, truncated, _ = env.step(action)

        # сохраняем награды и флаги терминальных состояний:
        memory.rewards.append(reward)
        memory.is_terminals.append(terminated)

        # выполняем обновление
        if timestep % update_timestep == 0:
            ppo.update(memory)
            memory.clear_memory()
            timestep = 0

        running_reward += reward
        if render:
            env.render()
        if terminated or truncated:
            break

    avg_length += t

    # останавливаемся, если avg_reward > solved_reward
    if running_reward > (log_interval * solved_reward):
        print("########## Принято! ##########")
        torch.save(ppo.policy.state_dict(), './PPO_{}.pth'.format(env_name))
        break

    # логирование
    if i_episode % log_interval == 0:
        avg_length = int(avg_length / log_interval)
        running_reward = int((running_reward / log_interval))

        print('Episode {} \t avg length: {} \t reward: {}'.format(i_episode, avg_length, running_reward))
        running_reward = 0
        avg_length = 0